# 개선점

### 1. StratifiedKFold 적용해 성능 비교
### 2. lgbm, xgboost, catboost 앙상블 모델 생성 (가중치 -> f1 macro)
### 3. BSNO 추가

# 0. 데이터셋 제작

In [ ]:
!pip install joblib

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install catboost

In [ ]:
!pip install tqdm

In [ ]:
import os
import numpy as np
import pandas as pd
import glob
import zipfile
import joblib
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from tqdm import tqdm
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm


os.getcwd()

In [3]:
df_test=pd.read_csv('./stack_value_23_test.csv')

df_test.fillna('NA', inplace=True)

In [4]:
df_test.drop(columns=["HSCD_2"], inplace=True)

In [ ]:
df_test

In [ ]:
df_test_hscd_bsno=df_test[['HSCD', 'BSNO']]
df_test_hscd_bsno.drop_duplicates(inplace=True)

In [ ]:
bsno_hscd_dict={}

for _, row in tqdm(df_test_hscd_bsno.iterrows()):
    key=row['BSNO']
    value=row['HSCD']
    if key not in bsno_hscd_dict:
        bsno_hscd_dict[key]=[]
    if value not in bsno_hscd_dict[key]:
        bsno_hscd_dict[key].append(value)

In [ ]:
df_test_hscd_cty=df_test[['HSCD', 'COUNTRYCD']]
df_test_hscd_cty.drop_duplicates(inplace=True)

In [ ]:
hscd_cty_dict={}

for _, row in tqdm(df_test_hscd_cty.iterrows()):
    key=row['HSCD']
    value=row['COUNTRYCD']
    if key not in hscd_cty_dict:
        hscd_cty_dict[key]=[]
    if value not in hscd_cty_dict[key]:
        hscd_cty_dict[key].append(value)  

In [ ]:
df_test_bsno_cty=df_test[['BSNO','COUNTRYCD']]
df_test_bsno_cty.drop_duplicates(inplace=True)

In [ ]:
bsno_cty_dict={}

for _, row in tqdm(df_test_bsno_cty.iterrows()):
    key=row['BSNO']
    value=row['COUNTRYCD']
    if key not in bsno_cty_dict:
        bsno_cty_dict[key]=[]
    if value not in bsno_cty_dict[key]:
        bsno_cty_dict[key].append(value)  

In [ ]:
bsno_hscd_agg_dict={}

for _, row in tqdm(df_test.iterrows()):
    key=str(row['BSNO'])+'_'+str(row['HSCD'])
    value=[row['전세계_해당품목_수출액'], row['수출국가수_해당품목']]
    if key not in bsno_hscd_agg_dict:
        bsno_hscd_agg_dict[key]=[]
        bsno_hscd_agg_dict[key].append(value)  

In [ ]:
bsno_cty_agg_dict={}

for _, row in tqdm(df_test.iterrows()):
    key=str(row['BSNO'])+'_'+str(row['COUNTRYCD'])
    value=[row['해당국_전체품목_수출액'], row['수출품목수_해당국']]
    if key not in bsno_cty_agg_dict:
        bsno_cty_agg_dict[key]=[]
        bsno_cty_agg_dict[key].append(value)  

In [ ]:
bsno_agg_dict={}

for _, row in tqdm(df_test.iterrows()):
    key=str(row['BSNO'])
    value=[row['전세계_전체품목_수출액'], row['수출품목수_전세계'], row['수출국가수_전체품목']]
    if key not in bsno_agg_dict:
        bsno_agg_dict[key]=[]
        bsno_agg_dict[key].append(value)  

In [ ]:
bsno_hscd_cty_dup_dict={}


for _, row in tqdm(df_test.iterrows()):
    key=str(row['BSNO'])+'_'+str(row['HSCD'])
    value=[row['COUNTRYCD']]
    if key not in bsno_hscd_cty_dup_dict:
        bsno_hscd_cty_dup_dict[key]=[value]
    if value not in bsno_hscd_cty_dup_dict[key]:
        bsno_hscd_cty_dup_dict[key].append(value)   

In [ ]:
df_test_new=df_test.copy()
new_df=pd.DataFrame()

for i, (bsno, hscd_list) in tqdm(enumerate(bsno_hscd_dict.items())):
    
    
    all_hscd_world_list=[]
    all_hscd_cty_list=[]
    cnt_hscd_world_list=[]
    cnt_cty_all_hscd_list=[]
    hscd_world_list=[]
    cnt_cty_hscd_list=[]
    cnt_hscd_cty_list=[]
    hscd_list=[]
    cty_list=[]
    
    
    print(bsno_agg_dict[f'{bsno}'])
    all_hscd_world=bsno_agg_dict[f'{bsno}'][0][0] # bsno 같으면 같음
    cnt_hscd_world=bsno_agg_dict[f'{bsno}'][0][1] # bsno 같으면 같음
    cnt_cty_all_hscd=bsno_agg_dict[f'{bsno}'][0][2]
    
    for hscd in bsno_hscd_dict[bsno]:
        
        
        hscd_world=bsno_hscd_agg_dict[f'{bsno}_{hscd}'][0][0]
        cnt_cty_hscd=bsno_hscd_agg_dict[f'{bsno}_{hscd}'][0][1]

        cty_len=0
        
        
        
        for cty in [c for c in hscd_cty_dict[hscd] if c not in bsno_hscd_cty_dup_dict[f'{bsno}_{hscd}']]:
            try:
                cnt_hscd_cty=bsno_cty_agg_dict[f'{bsno}_{cty}'][0][1]
                all_hscd_cty=bsno_cty_agg_dict[f'{bsno}_{cty}'][0][0]
                
            
                all_hscd_cty_list.append(all_hscd_cty)
                cnt_cty_hscd_list.append(cnt_cty_hscd)
                cnt_hscd_cty_list.append(cnt_hscd_cty)
                cty_list.append(cty)
                cty_len+=1
            except:
                continue
        
        hscd_world_list.extend([hscd_world]*cty_len)
        hscd_list.extend([hscd]*cty_len)
        
    all_hscd_world_list=[all_hscd_world]*len(cty_list)
    cnt_hscd_world_list=[cnt_hscd_world]*len(cty_list)
    cnt_cty_all_hscd_list=[cnt_cty_all_hscd]*len(cty_list)

    append_df = pd.DataFrame({'YEAR' : [2023]*len(cty_list), 'COUNTRYCD': cty_list, 
                               'HSCD':hscd_list, 
                               'BSNO' : bsno*len(cty_list),
                                '해당국_해당품목_수출액' : [0]*len(cty_list), 
                               '전세계_해당품목_수출액' : hscd_world_list, 
                               '전세계_전체품목_수출액' : all_hscd_world_list,
                                '해당국_전체품목_수출액' : all_hscd_cty_list, 
                               '수출품목수_전세계' : cnt_hscd_world_list, 
                               '수출국가수_전체품목' : cnt_cty_all_hscd_list,
                                '수출품목수_해당국' : cnt_hscd_cty_list, 
                               '수출국가수_해당품목': cnt_cty_hscd_list })

    new_df.append(append_df, ignore_index=True)
    if i%1000==0:
        print(bsno, ' finished!')

df_test.append(new_df, ignore_index=True)
df_test_new.to_csv('./전처리데이터/stack_value_23_test_new.csv')

df_test_new=df_test.copy()
new_df=pd.DataFrame()

for (bsno, hscd_list) in tqdm(bsno_hscd_dict.items()):
    
    
    all_hscd_world_list=[]
    all_hscd_cty_list=[]
    cnt_hscd_world_list=[]
    cnt_cty_all_hscd_list=[]
    hscd_world_list=[]
    cnt_cty_hscd_list=[]
    cnt_hscd_cty_list=[]
    hscd_list=[]
    cty_list=[]
    
    bsno_df=df_test[df_test['BSNO']==bsno]

    all_hscd_world=bsno_df['전세계_전체품목_수출액'].values[0] # bsno 같으면 같음
    cnt_hscd_world=bsno_df['수출품목수_전세계'].values[0] # bsno 같으면 같음
        
    hscd_len=len(bsno_hscd_dict[bsno])
    
    for hscd in bsno_hscd_dict[bsno]:
        
        
        hscd_bsno_df=bsno_df[bsno_df['HSCD']==hscd]
        hscd_world=hscd_bsno_df['전세계_해당품목_수출액'].values[0]
        cnt_cty_all_hscd=hscd_bsno_df['수출국가수_해당품목'].values[0]

        cnt_cty_hscd=hscd_bsno_df['COUNTRYCD'].nunique()
        cty_len=0
        
        
        print([c for c in hscd_cty_dict[hscd] if c not in hscd_bsno_df['COUNTRYCD'].unique()])
        
        for cty in [c for c in hscd_cty_dict[hscd] if c not in hscd_bsno_df['COUNTRYCD'].unique()]:
            try:
                cty_bsno_df=bsno_df[bsno_df['COUNTRYCD']==cty]
                cnt_hscd_cty=len(cty_bsno_df)
                all_hscd_cty=cty_bsno_df['해당국_해당품목_수출액'].sum()
                
            
                all_hscd_cty_list.append(all_hscd_cty)
                cnt_cty_hscd_list.append(cnt_cty_hscd)
                cnt_hscd_cty_list.append(cnt_hscd_cty)
                cty_list.append(cty)
                cty_len+=1
            except:
                continue
        
        hscd_world_list.extend([hscd_world]*cty_len)
        hscd_list.extend([hscd]*cty_len)
        
    all_hscd_world_list=[all_hscd_world]*len(cty_list)
    cnt_hscd_world_list=[cnt_hscd_world]*len(cty_list)
    cnt_cty_all_hscd_list=[cnt_cty_all_hscd]*len(cty_list)

    append_df = pd.DataFrame({'YEAR' : [2023]*len(cty_list), 'COUNTRYCD': cty_list, 
                               'HSCD':hscd_list, 
                               'BSNO' : bsno*len(cty_list),
                                '해당국_해당품목_수출액' : [0]*len(cty_list), 
                               '전세계_해당품목_수출액' : hscd_world_list, 
                               '전세계_전체품목_수출액' : all_hscd_world_list,
                                '해당국_전체품목_수출액' : all_hscd_cty_list, 
                               '수출품목수_전세계' : cnt_hscd_world_list, 
                               '수출국가수_전체품목' : cnt_cty_all_hscd_list,
                                '수출품목수_해당국' : cnt_hscd_cty_list, 
                               '수출국가수_해당품목': cnt_cty_hscd_list })
    display(append_df)
    new_df.append(append_df, ignore_index=True)
    print(bsno, ' finished!')

df_test.append(new_df, ignore_index=True)
df_test_new.to_csv('./전처리데이터/stack_value_23_test_new.csv')